# Conversion of tif files to HDF5 file

In [1]:
# Import necessary modules - need to use the "Tomorec Kernel"
import dxchange
import numpy as np
import h5py

In [2]:
# Create a directory into which the "wget" command will be run, and place that 
# directory name here:
dirname = '../data/'

# Go into that directory and run:
#   wget -m -np https://doi2.psi.ch/datasets/sls/X02DA/Data10/e17068/disk1/h11913_4_3_/tif
# After waiting 5 to 6 hours, the following directory structure will exist, where the
# "tif" directory will contain 2031 .tif files start with names that follow the 
# pattern "h11913_4_3_xxxx.tif"
doipath = 'doi2.psi.ch/datasets/sls/X02DA/Data10/e17068/disk1/h11913_4_3_/tif/'

# For this conversion script, the beginning of the file names are needed.
fname = dirname + doipath + 'h11913_4_3_'

# Specify where to write the output HDF5 file. Make sure atleast 16 GB of space 
# is available where this will be written.
foutname = dirname + 'hdf5-conversion/h11913_4_3.h5'

In [3]:
# Setup for the conversion
size_proj = [1801, 1562, 2560]
size_flat = [100, 1562, 2560]
size_dark = [30, 1562, 2560]
sproj = 128 # number of slices that fit to operating memory

In [4]:
# Create the output file in the location specified above
fid = h5py.File(foutname, mode='w')

# Create the structure within the file
projh5 = fid.create_dataset('exchange/data', size_proj, dtype='uint16')
flath5 = fid.create_dataset('exchange/data_flat', size_flat, dtype='uint16')
darkh5 = fid.create_dataset('exchange/data_dark', size_dark, dtype='uint16')

In [5]:
# Loop over the .tif image files, write to the HDF5 output file
# This should take 5 to 10 mintues
for k in range(0, np.int(np.ceil(size_proj[1] / sproj))):
   start = k*sproj
   end = min((k + 1) * sproj, size_proj[1])
   print("converting slices %d .. %d" % (start, end))
   proj, flat, dark = dxchange.read_sls_tomcat(fname, sino=(start, end))
   projh5[:, start:end, :] = proj
   flath5[:, start:end, :] = flat
   darkh5[:, start:end, :] = dark

fid.close()

/home/jasbru/.conda/envs/tomorec-kernel-environment/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


converting slices 0 .. 128
converting slices 128 .. 256
converting slices 256 .. 384
converting slices 384 .. 512
converting slices 512 .. 640
converting slices 640 .. 768
converting slices 768 .. 896
converting slices 896 .. 1024
converting slices 1024 .. 1152
converting slices 1152 .. 1280
converting slices 1280 .. 1408
converting slices 1408 .. 1536
converting slices 1536 .. 1562
